# ACTIVIDAD 3: Deep Learning para Clasificación de Texto

Edison Giraldo<br>
Joel Orellana<br>
Ruben Aponte

En esta actividad vamos a trabajar en clasificar textos. Se recorrerá todo el proceso desde traer el dataset hasta proceder a dicha clasificación. Durante la actividad se llevarán a cabo muchos procesos como la creación de un vocabulario, el uso de embeddings y la creación de modelos.

Las cuestiones presentes en esta actividad están basadas en un Notebook creado por François Chollet, uno de los creadores de Keras y autor del libro "Deep Learning with Python". 

En este Notebook se trabaja con el dataset "Newsgroup20" que contiene aproximadamente 20000 mensajes que pertenecen a 20 categorías diferentes.

El objetivo es entender los conceptos que se trabajan y ser capaz de hacer pequeñas experimentaciones para mejorar el Notebook creado.

In [1]:
#Basado en:
#https://keras.io/examples/nlp/pretrained_word_embeddings/

#Librerías

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Descarga de Datos

In [3]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

In [4]:
import os
import pathlib

#Estructura de directorios del dataset
data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

Number of directories: 20
Directory names: ['rec.sport.hockey', 'soc.religion.christian', 'sci.space', 'rec.autos', 'comp.graphics', 'talk.politics.misc', 'comp.windows.x', 'misc.forsale', 'sci.crypt', 'talk.politics.guns', 'alt.atheism', 'talk.religion.misc', 'comp.os.ms-windows.misc', 'comp.sys.mac.hardware', 'talk.politics.mideast', 'comp.sys.ibm.pc.hardware', 'sci.med', 'rec.sport.baseball', 'sci.electronics', 'rec.motorcycles']


In [5]:
#Algunos archivos de la categoria "com.graphics"
fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in comp.graphics: 1000
Some example filenames: ['38399', '39659', '38452', '38752', '38635']


In [6]:
#Ejemplo de un texto de la categoría "com.graphics"
print(open(data_dir / "comp.graphics" / "39625").read())

Path: cantaloupe.srv.cs.cmu.edu!rochester!udel!gatech!howland.reston.ans.net!newsserver.jvnc.net!castor.hahnemann.edu!hal.hahnemann.edu!brennan
From: brennan@hal.hahnemann.edu
Newsgroups: comp.graphics
Subject: .GIFs on a Tek401x ??
Date: 15 MAY 93 14:29:54 EST
Organization: Hahnemann University
Lines: 14
Message-ID: <15MAY93.14295461@hal.hahnemann.edu>
NNTP-Posting-Host: hal.hahnemann.edu


      I was skimming through a few gophers and bumped into one at NIH
   with a database that included images in .GIF format.  While I have
   not yet worked out the kinks of getting the gopher client to call
   an X viewer, I figure that the majority of the users here are not
   in an X11 environment - instead using DOS and MS-Kermit.

      With Kermit supporting Tek4010 emulation for graphics display,
   does anyone know of a package that would allow a Tek to display a
   .GIF image?  It would be of more use to the local population to
   plug something of this sort in as the 'picture' command in

<font color='green'>**Pregunta 1 (0.5 puntos): Escribe un texto de la categoría 'sci.electronics'**</font>

In [7]:
#Tu respuesta aqui
# seleccionando los filenames de 'sci.electronics':
fnames_sci = os.listdir(data_dir / "sci.electronics")
print("Number of files in sci.electronics: {}\n".format(len(fnames_sci)))
print("Some example filenames: {}\n".format(fnames_sci[:5]))
print("Select random filename in sci.electronics...\n")
rndm_filename = np.random.choice(fnames_sci)
print("A random example of sci.electronics:\n")
print(open(data_dir / "sci.electronics" / rndm_filename).read())

Number of files in sci.electronics: 1000

Some example filenames: ['53804', '53962', '53937', '54051', '53685']

Select random filename in sci.electronics...

A random example of sci.electronics:

Newsgroups: sci.electronics
Path: cantaloupe.srv.cs.cmu.edu!rochester!udel!bogus.sura.net!darwin.sura.net!zaphod.mps.ohio-state.edu!magnus.acs.ohio-state.edu!mbattist
From: mbattist@magnus.acs.ohio-state.edu (Mark A Battisti)
Subject: Re: Clear LCD or LED "sheets"
Message-ID: <1993Apr26.183654.29750@magnus.acs.ohio-state.edu>
Sender: news@magnus.acs.ohio-state.edu
Nntp-Posting-Host: magnusug.magnus.acs.ohio-state.edu
Organization: The Ohio State University
 writes:
References: <1993Apr26.141409.22154@magnus.acs.ohio-state.edu> <1rgssrINNnqb@ga
Distribution: usa
Date: Mon, 26 Apr 1993 18:36:54 GMT
Lines: 13

>>An ideal item would be an LED array for which each LED is about 1/2" square.
>>(Yes very coarse)  This is for distance viewing, but on a window.
>>Any pointers of suggestions would be mu

In [8]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

# Mezclando los datos para separarlos en Traning y Test

In [9]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

<font color='green'>**Pregunta 2 (0.5 puntos): ¿Por qué mezclamos los datos antes de separarlos en entrenamiento y validación?**</font>

**Tu respuesta aqui:**
Para evitar el sobreajuste (overfitting) de una determinada porción de categorías. Así, al mezclar los datos nos aseguramos que todas las categorías se encuentren representadas en el conjunto de datos de training, de esa manera evitamos el sesgo del modelo.

<font color='green'>**Pregunta 3 (1 punto): ¿Por qué estás seguro que en la división aleatoria cada muestra coincide con su etiqueta correcta?**</font>

**Tu respuesta aqui:** Porque al utilizar la misma semilla (1337) estamos asegurandonos que la mezcla aleatoria siga la misma permutación de los datos, por tanto los valores de samples y labels se permutan en el mismo orden en las listas.

# Tokenización de las palabras con TextVectorization 

In [10]:
from tensorflow.keras.layers import TextVectorization
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [11]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [12]:
len(vectorizer.get_vocabulary())

20000

<font color='green'>**Pregunta 4 (2 puntos): En la construcción del vocabulario hemos limitado el número de tokens a 20.000 ¿Podrías indicar el número de token diferentes o tamaño del vocabulario sin limitar el número de tokens? Es decir, ¿Cuántas palabras diferentes existen en los documentos procesados como instancias?**</font>

In [13]:
#Tu código para responder a esta pregunta aqui:
# Procedemos a crear un vectorizer 'without_limit' sin limitar el vocabulario
# luego adaptamos el texto de train_samples:
without_limit = TextVectorization(max_tokens=None, output_sequence_length=None)
text_ds_2 = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
# vectorizando el text_ds_2:
without_limit.adapt(text_ds_2)
# longitud del vocabulario en tokens:
print("The vocabulary has {} tokens\n".format(len(without_limit.get_vocabulary())))
voc_lst = without_limit.get_vocabulary()

# Definimos una función para verificar que no existen tokens repetidos: 
def repeated_elements(voc_lst):
    conjunto = set(voc_lst)
    if len(conjunto) < len(voc_lst):
        print("There are repeated elements")
    else:
        print("There are no repeated elements")

repeated_elements(voc_lst)

The vocabulary has 162113 tokens

There are no repeated elements


# Viendo la salida de Vectorizer

In [14]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 3451, 1697,   15,    2, 6678])

In [15]:
output

<tf.Tensor: shape=(1, 200), dtype=int64, numpy=
array([[   2, 3451, 1697,   15,    2, 6678,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   

In [16]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [17]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3451, 1697, 15, 2, 6678]

<font color='green'>**Pregunta 5 (1 punto): Arriba tenemos la codificación de la frase "the cat sat on the mat". Imagina la siguiente situación. La salida de vectorizer() para codificar los tokens ["El", "gato", "está", "sobre", "el", "tejado"] es la siguiente [1, 121, 405, 1, 45, 4561]. Si cada uno de los valores indica el índice en el que se encuentra cada palabra en el array creado para codificarla. ¿Podría ser correcta esta salida?**</font>

**Tu respuesta aqui:** La codificación mostrada es incorrecta, debido a que el vocabulario proviene de una tokenización, entonces las palabras "El" y "el" deberían tener la misma codificación y no la tienen, "El" es 1 y "el" es 45. También, teniendo en cuenta la codificación de "El" que es 1, en la cuarta posición está "sobre" que se codifica igual como 1, lo cual no es correcto y no tiene sentido tampoco.

# Tokenización de los datos de entrenamiento y validación

In [18]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

# Creación del modelo con un embedding hecho a mano y con redes neuronales convolucionales

Aunque las Redes Neuronales Convolucionales se desarrollaron originalmente para procesamiento de imágenes, se pueden aplicar a problemas de procesamiento de lenguaje natural gracias a su capacidad para capturar patrones locales y globales en los datos de entrada.

Previa a la alimentación de las capas convolucionales, vamos a incluir una capa de Embedding. Como hemos visto en clase, un embedding de palabras se refiere a una representación vectorial densa de una palabra en un espacio de características continuo de alta dimensión. Como resultado del embedding vamos capturar la semántica de las palabras de manera que palabras similares tengan representaciones vectoriales similares.

A continuación se incluye la creación de un modelo con un embedding hecho a mano y usando  Redes Neuronales Convolucionales

In [19]:
modeloEmbeddingManualConvolucionales = keras.models.Sequential()
modeloEmbeddingManualConvolucionales.add(keras.layers.Embedding(20000, 10, input_length=200))
modeloEmbeddingManualConvolucionales.add(keras.layers.Conv1D(128, 5, activation="relu"))
modeloEmbeddingManualConvolucionales.add(keras.layers.MaxPooling1D(5))
modeloEmbeddingManualConvolucionales.add(keras.layers.Conv1D(128, 5, activation="relu"))
modeloEmbeddingManualConvolucionales.add(keras.layers.MaxPooling1D(5))
modeloEmbeddingManualConvolucionales.add(keras.layers.Conv1D(128, 5, activation="relu"))
modeloEmbeddingManualConvolucionales.add(keras.layers.GlobalMaxPooling1D())
modeloEmbeddingManualConvolucionales.add(keras.layers.Dense(128, activation='relu'))
modeloEmbeddingManualConvolucionales.add(keras.layers.Dropout(0.3))
modeloEmbeddingManualConvolucionales.add(keras.layers.Dense(20, activation='softmax'))

In [20]:
modeloEmbeddingManualConvolucionales.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modeloEmbeddingManualConvolucionales.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
modeloEmbeddingManualConvolucionales.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 30s 156ms/step - loss: 2.9285 - acc: 0.0698 - val_loss: 2.6645 - val_acc: 0.1085
Epoch 2/20
125/125 [==============================] - 18s 142ms/step - loss: 2.5296 - acc: 0.1228 - val_loss: 2.4305 - val_acc: 0.1413
Epoch 3/20
125/125 [==============================] - 11s 85ms/step - loss: 2.2943 - acc: 0.1612 - val_loss: 2.1812 - val_acc: 0.2063
Epoch 4/20
125/125 [==============================] - 9s 69ms/step - loss: 2.1216 - acc: 0.1999 - val_loss: 2.0665 - val_acc: 0.2268
Epoch 5/20
125/125 [==============================] - 7s 54ms/step - loss: 1.9475 - acc: 0.2474 - val_loss: 1.9142 - val_acc: 0.2928
Epoch 6/20
125/125 [==============================] - 8s 61ms/step - loss: 1.7646 - acc: 0.3144 - val_loss: 1.8447 - val_acc: 0.3071
Epoch 7/20
125/125 [==============================] - 4s 29ms/step - loss: 1.5633 - acc: 0.3852 - val_loss: 1.6393 - val_acc: 0.4184
Epoch 8/20
125/125 [==============================] - 4s 34ms/st

<font color='green'>**Pregunta 6 (1 punto): ¿Por qué usamos convoluciones de 1 dimensión para procesar texto?**</font>

**Tu respuesta aqui:** A diferencia de los modelos de redes neuronales convolucionales para imagenes en los que se usa Conv2D, debido a que las imágenes son de dos dimensiones y los datos de textos están en una sola dimensión, conocidos como sequencias de caracteres, por tal razón se usan redes convolucionales en 1D. Las redes convolucionales en 1D también se pueden usar en series de tiempo.

#Creación del modelo con un embedding hecho a mano con redes neuronales clásicas





<font color='green'>**Pregunta 7 (2 puntos): Crea un nuevo modelo partiendo del model anterior pero en lugar de Redes Neuronales Convolucionales vamos a utilizar Redes Neuronales Clásicas. Para ello, tras la capa Embedding añade una capa Flatten, una capa densa de 512 neuronas con función de activación relu y una capa Dropout de regulización al 30%. Por último, no olvides incluir la capa de salida con tantas neuronas como clases haya y la función de activación softmax.**</font>

**Tu respuesta aqui:** Se crea un modelo secuencial, con la primera capa Embedding con tamaño de vocavulario 20000, tamaño de vector de embedding 10 y longitud de las secuencias de entrada 200. posteriormente, una capa densa de 512 nueronas con función de activación ReLU y una capa dropout de 30%. Finalmente la capad de salida para entregar el vector de probabilidades de las 20 categorías.

In [21]:
# Tu código aqui:
modeloEmbeddingManualClasicas = keras.models.Sequential()
modeloEmbeddingManualClasicas.add(keras.layers.Embedding(20000, 10, input_length=200))
modeloEmbeddingManualClasicas.add(keras.layers.Flatten())
modeloEmbeddingManualClasicas.add(keras.layers.Dense(512, activation='relu'))
modeloEmbeddingManualClasicas.add(keras.layers.Dropout(0.3))
modeloEmbeddingManualClasicas.add(keras.layers.Dense(20, activation='softmax'))

In [22]:
modeloEmbeddingManualClasicas.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modeloEmbeddingManualClasicas.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
modeloEmbeddingManualClasicas.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))
print(modeloEmbeddingManualClasicas.summary())

Epoch 1/20
125/125 [==============================] - 14s 106ms/step - loss: 2.8812 - acc: 0.1044 - val_loss: 2.6420 - val_acc: 0.1615
Epoch 2/20
125/125 [==============================] - 9s 76ms/step - loss: 2.2863 - acc: 0.2590 - val_loss: 2.0703 - val_acc: 0.3003
Epoch 3/20
125/125 [==============================] - 8s 61ms/step - loss: 1.6822 - acc: 0.4592 - val_loss: 1.6473 - val_acc: 0.4281
Epoch 4/20
125/125 [==============================] - 7s 53ms/step - loss: 1.1654 - acc: 0.6478 - val_loss: 1.3265 - val_acc: 0.5641
Epoch 5/20
125/125 [==============================] - 5s 40ms/step - loss: 0.7902 - acc: 0.7732 - val_loss: 1.1442 - val_acc: 0.6164
Epoch 6/20
125/125 [==============================] - 5s 42ms/step - loss: 0.5500 - acc: 0.8421 - val_loss: 1.0643 - val_acc: 0.6374
Epoch 7/20
125/125 [==============================] - 4s 36ms/step - loss: 0.3913 - acc: 0.8916 - val_loss: 1.0377 - val_acc: 0.6574
Epoch 8/20
125/125 [==============================] - 3s 20ms/step 

Tras hacer este experimento, es posible que pase una cosa curiosa y es que la red totalmente conectada tenga un mejor comportamiento que la convolucional. 
En general, las redes neuronales convolucionales tienden a superar a las totalmente conectadas en la mayoría de los problemas de este tipo debido a su capacidad para capturar patrones espaciales y posicionales en los datos de entrada. Dicho esto, en algunos casos, una red neuronal clásica podría superar a una convolucional en un problema de procesamiento de lenguaje natural, especialmente si el conjunto de datos es pequeño y las características relevantes son relativamente simples. En este caso, la mayoría de las categorías puede ser que tengan un vocabulario específico y esa pueda ser una de las razones. 





#Creación del modelo con el embedding GloVe y redes neuronales convolucionales


Esta vez, en vez de crear nosotros mismos el embedding, vamos a utilizar Glove. Los vectores de palabras generados por GloVe capturan las relaciones semánticas y sintácticas entre las palabras en un corpus de texto mucho más grande que el nuestro. 

In [23]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-06-10 17:51:11--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-06-10 17:51:11--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-06-10 17:51:11--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

<font color='green'>**Pregunta 8 (1 puntos): 	El embedding que usa la actividad es el glove.6B. ¿De dónde se han obtenido los textos que se han usado para entrenarlo? ¿Cántos WordVectors lo componen? ¿De cuántas dimensiones tiene versiones? Puedes ayudarte del link https://nlp.stanford.edu/projects/glove/ para buscar la información**</font>




**Tu respuesta aqui:** Los textos utilizados para GloVe.6B son los siguientes:<br>
* Wikipedia 2014 en diferentes idiomas.
* Gigaword 5: está compuesto por artículos de noticias y textos de diferentes fuentes y dominios, recopilados a lo largo de varios años hasta dicimebre de 2010. Está compuesto de 6B tokens y de 400k word vectors (vocabulario). Cuenta con versiones de 50d, 100d, 200d, & 300d vectors.

In [24]:
path_to_glove_file = os.path.join(
    os.path.expanduser("~"), ".keras/datasets/glove.6B.100d.txt"
)
path_to_glove_file = "glove.6B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


Creamos la capa Embedding de keras. Se trata de una matrix numpy donde el valor de cada posición corresponde con el vector pre-entreneado del vocabulario tras el vectorizer

In [25]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 17954 words (2046 misses)


Cargamos el embedding como una capa keras. Al poner trainable=False nos quedamos con los valores del modelo pre-entrenado, es decir, no actualizamos estos valores a lo largo del entrenamiento

In [26]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [27]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
modelEmbeddingGloveConvolucionales = keras.Model(int_sequences_input, preds)
modelEmbeddingGloveConvolucionales.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 100)         2000200   
                                                                 
 conv1d_3 (Conv1D)           (None, None, 128)         64128     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, None, 128)        0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, None, 128)        0         
 1D)                                                         

In [28]:
modelEmbeddingGloveConvolucionales.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
modelEmbeddingGloveConvolucionales.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))
predictions = modelEmbeddingGloveConvolucionales.predict(x_val)

Epoch 1/20
125/125 [==============================] - 3s 13ms/step - loss: 2.7018 - acc: 0.1264 - val_loss: 2.2242 - val_acc: 0.2343
Epoch 2/20
125/125 [==============================] - 1s 9ms/step - loss: 2.0104 - acc: 0.3101 - val_loss: 1.6258 - val_acc: 0.4424
Epoch 3/20
125/125 [==============================] - 1s 9ms/step - loss: 1.5472 - acc: 0.4697 - val_loss: 1.2912 - val_acc: 0.5596
Epoch 4/20
125/125 [==============================] - 1s 9ms/step - loss: 1.2982 - acc: 0.5562 - val_loss: 1.2218 - val_acc: 0.5724
Epoch 5/20
125/125 [==============================] - 1s 9ms/step - loss: 1.1288 - acc: 0.6107 - val_loss: 1.1361 - val_acc: 0.6122
Epoch 6/20
125/125 [==============================] - 1s 8ms/step - loss: 0.9959 - acc: 0.6614 - val_loss: 1.0279 - val_acc: 0.6457
Epoch 7/20
125/125 [==============================] - 1s 8ms/step - loss: 0.8874 - acc: 0.6908 - val_loss: 1.0778 - val_acc: 0.6379
Epoch 8/20
125/125 [==============================] - 1s 8ms/step - loss: 0

#Creación del modelo final


<font color='green'>**Pregunta 9 (1 puntos): 	A estas alturas ya te habrás dado cuenta que los Transformer que has utilizado en la asignatura de Procesamiento de Lenguaje Natural son, probablemente, los que mejores resultados obtienen. Sin utilizar Transformer, crea un modelo tú, bien ajustando los hiperparámetros de los modelos anteriores o uno completamente nuevo, que llegue al menos al 72% accuracy**</font>

**Tu respuesta aqui:** Es este caso, se utliza el modelo anteriormente planteado pero adicionando una capa LSTM con 256 parámetros de entrada después de la capa de Embedding. Esto, permite al modelo aprender y utilizar patrones más complejos en la secuencia de entrada, capturando dependencias temporales a largo plazo. También, se modificaron la cantidad de parametros a las dos primeras capas convolucionales de 128 a 256.

In [48]:
# Tu codigo aqui:
# Se importa LSTM:
from tensorflow.keras.layers import LSTM

int_sequences_input_2 = keras.Input(shape=(None,), dtype="int64")
embedded_sequences1 = embedding_layer(int_sequences_input_2)
x1 = layers.Conv1D(256, 5, activation="relu")(embedded_sequences1)
x1 = layers.MaxPooling1D(5)(x1)
x1 = layers.Conv1D(256, 5, activation="relu")(x1)
x1 = layers.MaxPooling1D(5)(x1)
x1 = layers.Conv1D(128, 5, activation="relu")(x1)
x1 = layers.GlobalMaxPooling1D()(x1)
# Se añade la capa LSTM con capacidad para procesar información con 256 estados:
x1 = layers.Reshape((-1, 128))(x1)
x1 = LSTM(256)(x1)
x1 = layers.Dense(128, activation="relu")(x1)
x1 = layers.Dropout(0.5)(x1)
preds1 = layers.Dense(len(class_names), activation="softmax")(x1)
modelEmbeddingGloveConvolucionales1 = keras.Model(int_sequences_input_2, preds1)
modelEmbeddingGloveConvolucionales1.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 100)         2000200   
                                                                 
 conv1d_34 (Conv1D)          (None, None, 256)         128256    
                                                                 
 max_pooling1d_18 (MaxPoolin  (None, None, 256)        0         
 g1D)                                                            
                                                                 
 conv1d_35 (Conv1D)          (None, None, 256)         327936    
                                                                 
 max_pooling1d_19 (MaxPoolin  (None, None, 256)        0         
 g1D)                                                      

In [49]:
modelEmbeddingGloveConvolucionales1.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
modelEmbeddingGloveConvolucionales1.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))
predictions1 = modelEmbeddingGloveConvolucionales1.predict(x_val)

Epoch 1/20
125/125 [==============================] - 6s 28ms/step - loss: 2.7395 - acc: 0.1041 - val_loss: 2.4089 - val_acc: 0.1753
Epoch 2/20
125/125 [==============================] - 2s 15ms/step - loss: 2.1300 - acc: 0.2430 - val_loss: 1.8666 - val_acc: 0.3361
Epoch 3/20
125/125 [==============================] - 2s 15ms/step - loss: 1.7249 - acc: 0.3807 - val_loss: 1.4225 - val_acc: 0.4884
Epoch 4/20
125/125 [==============================] - 2s 16ms/step - loss: 1.4263 - acc: 0.4860 - val_loss: 1.2697 - val_acc: 0.5451
Epoch 5/20
125/125 [==============================] - 2s 16ms/step - loss: 1.2133 - acc: 0.5672 - val_loss: 1.1301 - val_acc: 0.5984
Epoch 6/20
125/125 [==============================] - 2s 15ms/step - loss: 1.0490 - acc: 0.6320 - val_loss: 1.0460 - val_acc: 0.6387
Epoch 7/20
125/125 [==============================] - 2s 17ms/step - loss: 0.9075 - acc: 0.6790 - val_loss: 1.0548 - val_acc: 0.6469
Epoch 8/20
125/125 [==============================] - 2s 15ms/step - 